In [4]:
import json
import os
import pandas as pd
import glob
import pandasgui as pdg

print(pdg.__version__)

0.2.13


In [7]:
dataset = ["OpenAPS", "OPENonOH"]
outdir = "/home/reinhold/Daten/OPEN/"
outfilename = f"duplicates_{dataset[0]}_{dataset[1]}.csv"

indir = [f"{outdir}/{d}_Data/csv_per_day/" for d in dataset]
infilename = [f"entries_{d}.csv" for d in dataset]

df = [pd.read_csv(os.path.join(indir[i], infilename[i]), header=0, parse_dates=[1], index_col=0) for i in range(2)]
df[0]["date"] = pd.to_datetime(df[0]["date"],format="%Y-%m-%d")
out = df[0].merge(df[1], left_on="date", right_on="date", how="inner", suffixes=("_OpenAPS", "_OPENonOH"))

print(df[0].columns)
for col in df[0].columns:
    if not "sgv" in col: continue
    out["diff_" + col] = 2*(out[col + "_OpenAPS"] - out[col + "_OPENonOH"])/(out[col + "_OpenAPS"] + out[col + "_OPENonOH"])

df2 = out[out.diff_sgv_mean*out.diff_sgv_mean< 1e-6]

gui = pdg.show(df2)

df2.to_csv(os.path.join(outdir, outfilename))
print(os.path.join(outdir, outfilename) + " created")


PandasGUI INFO — pandasgui.gui — Opening PandasGUI


Index(['date', 'sgv_mean', 'sgv_std', 'sgv_min', 'sgv_max', 'sgv_count',
       'filename', 'user_id', 'second_id'],
      dtype='object')
/home/reinhold/Daten/OPEN/duplicates_OpenAPS_OPENonOH.csv created
